# Рейтинг пользователей по количеству друзей в группах.

In [21]:
import collections
import vk
import pandas as pd
import time

In [22]:
VK_TOKEN = 'aa9a438374e5f7808961c523653ba7217c1d8efdba09cfadcdbde9f1a1906e8e970e03a403e119dcbe740'
ID = 10385605
session = vk.Session(VK_TOKEN)
vk_api = vk.API(session, v=5.103)
#очередь регулирует количество запросов в секунду 
deq = collections.deque(maxlen=4)

Метод считает медиану списка

In [23]:
def median(l):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    l = sorted(l)
    if len(l) % 2 == 1:
        return l[int(len(l) / 2)]
    else:
        return (l[int(len(l) / 2 - 1)] + l[int(len(l) / 2)]) / 2

Медиана списка, содержащего количество друзей для каждой группы пользователя

In [24]:
def get_med(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    count_friends =[]
    groups = vk_api.groups.get(user_id=id)
    for group in groups['items']:
        try:
            count_friends.append(vk_api.groups.getMembers(group_id=group, filter='friends')['count'])
        except Exception:
            continue
    return median(count_friends)

Метод составляет рейтинг друзей, основываясь на их медиане 

In [25]:
def get_my_rating(id):
    deq.appendleft(time.time())
    if len(deq) == 4:
        time.sleep(max(1 + deq[3] - deq[0], 0))
    friends = vk_api.friends.get(user_id=id, extended=1, fields='nickname')['items']
    df = pd.DataFrame(columns=['friend_name', 'median'])
    i = 0
    for friend in friends:
        try:
            med = get_med(friend['id'])
            friend_name = friend['first_name'] + ' ' + friend['last_name']
            df.loc[i] = [friend_name, int (med)]
            i += 1
        except Exception:

            continue
    df = df.sort_values(by='median', ascending=False)
    return df

In [26]:
df = get_my_rating(ID)
df

,friend_name,median
94,Анжелика Романова,42
99,Иван Брекоткин,38
22,Ринат Шигапов,29
76,Марсель Сидиков,24
49,Мансур Гарипов,24
...,...,...
47,Дарина Зайнуллина,1
51,Тоня Кудряшова,1
81,Максим Сандин,1
80,Карина Король,1
